## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель проекта** - исследование данных о клиентах банка для получения информации о том, влияют ли на отсутствие просроченной задолженности различные параметры:
- цель кредита 
- ежемесячный доход заемщика
- его семейное положение
- количество детей

Массив данных о заемщиках содержит в себе информацию о возрасте клиента, его трудовом стаже, образовании, семейном положении, доходе, количестве детей, типе занятости, ежемесячном доходе, факте наличия просроченной задолженности, цели кредитования.

Чтобы дать ответ на поставленные вопросы, сперва требуется обработать данные для дальнейшей работы с ними. Для этого решается ряд **задач**:

- заполнение пропусков в столбцах 
- изменение типа данных в столбцах
- обработка дубликатов: проведение проверки, повторяются ли в таблице одинаковые заемщики; изменение регистра в столбцах 
- категоризация (по целям кредитования, доходу, количеству детей)

## Содержание

[Обзор данных](#review)

[Предобработка данных](#pretreatment)

- [Обработка пропусков](#omissions)

- [Замена типов данных](#substitution)

- [Обработка дубликатов](#duplicate_handling)

- [Лемматизация](#lemmatization)

- [Категоризация данных](#categorization)

[Ответы на поставленные вопросы](#answers)

[Общий вывод исследования](#conclusion)



### Изучение общей информации <a name="review"></a>

Импорт библиотек, которые потребуются для выполнения исследования:

In [1]:
import pandas as pd
from pymystem3 import Mystem

Считывание файла с данными в датафрейм **borrowers**:

In [3]:
borrowers = pd.read_csv('/datasets/data.csv')

Получение данных о первых 15 строках таблицы:

In [4]:
borrowers.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Общая информация о данных таблицы:

In [5]:
borrowers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Рассмотрим полученную информацию.

В таблице 12 столбцов:

- **children** — количество детей в семье / целочисл. тип данных
- **days_employed** — общий трудовой стаж в днях / числа с плав.точкой
- **dob_years** — возраст клиента в годах / целочисл. тип данных 
- **education** — уровень образования клиента / строковый тип данных
- **education_id** — идентификатор уровня образования / целочисл. тип данных
- **family_status** — семейное положение / строковый тип данных
- **family_status_id** — идентификатор семейного положения / целочисл. тип данных
- **gender** — пол клиента / строковый тип данных
- **income_type** — тип занятости / строковый тип данных
- **debt** — имел ли задолженность по возврату кредитов / целочисл. тип данных
- **total_income** — ежемесячный доход / числа с плав.точкой
- **purpose** — цель получения кредита

Во всех столбцах, кроме **days_employed** и **total_income**, количество значений одинаково. В указанных столбцах имеются пропуски.

### Вывод

Каждая строка таблицы содержит сведения о заемщиках банка, взявших кредит, - доход, трудовой стаж, количество детей и пр. 

Проблемы, которые нужно решить, прежде чем приступить к поиску ответов на поставленные вопросы: 
- пропуски в столбцах **days_employed** и **total_income**
- изменение типа данных в столбце **days_employed** с вещественного на целочисленный
- обработка дубликатов: проведение проверки, повторяются ли в таблице одинаковые заемщики; изменение регистра в столбце **education** (и, при обнаружении, других)
- категоризация (по целям кредитования, доходу, количеству детей)


### Предобработка данных <a name="pretreatment"></a>

### Обработка пропусков <a name="omissions"></a>

Столбцы **days_employed** и **total_income** каждый содержат меньше значений (19351), чем все остальные (21525). Следовательно, в них имеются пропуски. Заметим, что общее количество значений в этих столбцах одинаково. Это позволяет предположить, что пропуски приходятся на одни и те же строки. Проверим гипотезу, подсчитав количество строк, в которых одновременно пропущены **days_employed** и **total_income**:


In [6]:
borrowers.loc[borrowers['days_employed'].isna() & borrowers['total_income'].isna()].shape[0]

2174

Таких строк 2174, а это и есть разница между 21525 и 19351. Таким образом, пропуски в столбцах **days_employed** и **total_income** действительно находятся в одних и тех же строках. Объяснения могут быть следующими: указаные строки либо описывают заемщиков, не работающих в текущий момент (нет ни стажа, ни ежемесячного дохода), либо заемщики являются типичными для выборки, а значения стажа и дохода отличаются от нулевых, но не сохранились по каким-то причинам.  

В первом случае осмысленно будет заменить пропущенные значения на нули, во втором - на средние значения показателя по выборке.

Проверим, сколько в таблице заемщиков с неуказанным доходом в разбивке по типам занятости:

In [7]:
borrowers[borrowers['total_income'].isna()]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Безработных в списке не оказалось, поэтому, предположительно, все лица с неуказанным доходом на самом деле его имеют. Ежемесячный доход - количественная переменная, и пропуски можно заменить на среднее арифметическое по выборке. Проверим, нет ли существенных выбросов, которые могут исказить среднее значение: 

In [8]:
borrowers['total_income'].min()

20667.26379327158

In [9]:
borrowers['total_income'].max()

2265604.028722744

Минимальный и максимальный доходы выглядят правдоподобно. Рассчитаем среднее:

In [10]:
mean_income = borrowers['total_income'].mean()

In [11]:
mean_income

167422.30220817294

При оценке заработной платы по выборке часто используют показатель медианы, чтобы избежать усреднения, если распределение не носит нормальный характер. Проверим, чему равна медиана:

In [12]:
median_income = borrowers['total_income'].median()

In [13]:
median_income

145017.93753253992

Значение медианы не сильно отличается от среднего арифметического, поэтому не принципиально, какой из показателей выбрать. Пусть это будет среднее арифметическое. Заменим на него пропуски в столбце **total_income**:

In [14]:
borrowers['total_income'] = borrowers['total_income'].fillna(mean_income)

Что касается показателя стажа, без преобразования данных сразу заменить пропуски на среднее арифметическое было бы неверно (хотя стаж - тоже количественная переменная). Во-первых, бросается в глаза значительное количество отрицательных значений: 

In [15]:
borrowers.loc[borrowers['days_employed'] < 0].shape[0]

15906

Наименьший отрицательный показатель стажа равен:

In [16]:
borrowers.loc[borrowers['days_employed'] < 0]['days_employed'].min()

-18388.949900568383

По абсолютному значению он соответствует примерно 50,37 годам стажа (18388 / 365). Остальные значения от -18388 до 0, взятые в абсолютном значении, логически соответствуют стажу от 0 до 50,37 лет.

Значительное количество отрицательных значений (15906 / 21525 * 100% = 73,8%) говорит скорее всего о том, что это техническая ошибка, устранить которую можно, умножив указанные показатели на -1:

In [17]:
borrowers.loc[borrowers['days_employed'] < 0, 'days_employed'] = borrowers.loc[borrowers['days_employed'] < 0]*(-1)

Во-вторых, в столбце **days_employed** встречаются значения, не соответствующие логике переменной. Так, заемщик из пятой строки имеет указанный показатель, равный 340266.072047, что эквивалентно 932 годам стажа. Очевидно, что трудовой стаж не может превышать максимального возраста среди всех заемщиков:

In [18]:
max_age = borrowers['dob_years'].max()

In [19]:
max_age

75

75 лет - это примерно 27375 дней. Рассмотрим, в каких границах лежит недостоверные значения столбца **days_employed**: 

In [20]:
borrowers.loc[borrowers['days_employed'] > 27375]['days_employed'].min()

328728.72060451825

In [21]:
borrowers.loc[borrowers['days_employed'] > 27375]['days_employed'].max()

401755.40047533

Заметим, что если разделить полученные числа на 24, получим примерно 13697 и 16740, что соответствует ожидаемым значениям показателя. Это позволяет предположить, что трудовой стаж части заемщиков был представлен не в днях, а в часах. Таким образом, разделив на 24 все значения, большие 328728, приведем данные к нужной форме:

In [22]:
borrowers.loc[borrowers['days_employed'] > 328728, 'days_employed'] = borrowers.loc[borrowers['days_employed'] > 328728, 'days_employed'] / 24

Теперь данные столбца days_employed приведены к единому виду, и можно заполнить пропуски средним значением трудового стажа:

In [23]:
mean_daya_employed = borrowers['days_employed'].mean()

In [24]:
mean_daya_employed

4641.641176180656

In [25]:
borrowers['days_employed'] = borrowers['days_employed'].fillna(mean_daya_employed)

Убедимся, что пропущенных значений в таблице не осталось:

In [26]:
borrowers.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

В таблице были пропущены данные о стаже и ежемесячном доходе некоторых заемщиков. Пропуски находились в одних и тех же строках. Отсутсвующие данные были заменены средними значениями показателей по выборке (в случае стажа - после приведения данных к корректному виду).

### Замена типа данных <a name="substitution"></a>

Заметим, что в столбце **days_employed** тип данных не соответствует реальному положению вещей: количество дней стажа должно быть целым числом. 

Приведем значения к нужному типу (int) с помощью метода **astype()**:

In [27]:
borrowers['days_employed'] = borrowers['days_employed'].astype('int')

Для этих же целей можно было использовать метод **to_numeric()**, однако принципиальной разницы в указанном случае нет.

### Вывод

Тип данных показателя трудового стажа **days_employed** в исходной таблице был изменен с вещественного на целочисленный в соответствии с логикой переменной. Выбор метода обусловлен личными предпочтениями автора.

### Обработка дубликатов <a name="duplicate_handling"></a>

Данные, однозначно идентифицирующие заемщика (например, ИНН, номер паспорта или некий внутрибанковский ID клиента), в приведенной таблице не представлены. Для поиска повторяющихся заемщиков попробуем использовать показатель дохода, т.к. он приведен с большой точностью и, возможно, обеспечит малое число повторов:



In [28]:
borrowers['total_income'].value_counts()

1.674223e+05    2174
1.128744e+05       1
1.091628e+06       1
1.820367e+05       1
1.224220e+05       1
                ... 
1.796527e+05       1
1.332992e+05       1
1.150808e+05       1
8.489678e+04       1
1.500141e+05       1
Name: total_income, Length: 19352, dtype: int64

Поскольку именно в 2174 строках пропущенные значения столбца **total_income** были искусственно заменены на среднее арифметическое, а других повторов нет, можно предположить, что и сами заемщики в таблице не повторяются.

Теперь рассмотрим все категориальные переменные на предмет поиска дубликатов. В столбце education одинаковые по смыслу значения написаны разным регистром, что помешает дальнейшему анализу. Исправим это с помощью метода **str.lower()**:

In [29]:
borrowers['education'] = borrowers['education'].str.lower()

In [30]:
borrowers['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

По аналогии проверим остальные столбцы с категориальными переменными на предмет повторяющихся значений, отличающихся только регистром.

В столбце **family_status** такой проблемы нет:

In [31]:
borrowers['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

В столбце **gender** тоже, однако у одного из заемщиков не определен пол:

In [32]:
borrowers['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Поскольку одна строка существенно не повлияет на результаты исследования, и, более того, в задаче не требуется исследовать зависимость наличия просрочки по кредитам от пола, поменяем пол на мужской:

In [33]:
borrowers.loc[borrowers['gender'] == 'XNA', 'gender'] = 'M'

In [34]:
borrowers['gender'].value_counts()

F    14236
M     7289
Name: gender, dtype: int64

В столбце **income_type** нет значений, отличающихся только регистром:

In [35]:
borrowers['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

В столбце **purpose** также нет значений, отличающихся только регистром, однако встречаются разные формулировки цели кредитования, отличающиеся по форме, но не по смыслу (например, "свадьба" и "на проведение свадьбы"). Эта проблема будет решена в следующем разделе.

In [36]:
borrowers['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

### Вывод

Дублирующихся строк в таблице не обнаружено. Исправлены дубликаты - одинаковые слова, написанные разным регистром - в столбце **education**.

### Лемматизация <a name="lemmatization"></a>

Как было замечено выше, необходимо сделать изменения в столбце purpose: привести к единому виду все цели кредитования, имеющий одинаковый смысл, но по-разному записанные. Будем использовать для этого лемматизацию с помощью методов библиотеки **PyMystem**.

In [37]:
m = Mystem()

In [38]:
lemmas = []

In [39]:
for purpose in borrowers['purpose']:
    lemmas += m.lemmatize(purpose)

Посмотрим на повторы лемм:

In [40]:
from collections import Counter
print(Counter(lemmas))

Counter({' ': 33677, '\n': 21525, 'недвижимость': 6367, 'покупка': 5912, 'жилье': 4473, 'автомобиль': 4315, 'образование': 4022, 'с': 2924, 'операция': 2610, 'свадьба': 2348, 'свой': 2235, 'на': 2233, 'строительство': 1881, 'высокий': 1375, 'получение': 1316, 'коммерческий': 1315, 'для': 1294, 'жилой': 1233, 'сделка': 944, 'дополнительный': 909, 'заниматься': 908, 'проведение': 777, 'сыграть': 774, 'сдача': 653, 'семья': 641, 'собственный': 635, 'со': 630, 'ремонт': 612, 'подержанный': 489, 'подержать': 479, 'приобретение': 462, 'профильный': 436})


Лемматизировав и изучив значения, которые принимает показатель цели кредитования, можно выделить 4 основные группы:
- на свадьбу
- на операции с недвижимостью
- на приобретение автомобиля
- на получение образования

    Создадим новые столбцы в исходной таблице: в **lemmas** поместим все леммы столбца **purpose**, в **purpose_briefly** - одну из четырех целей кредитования в зависимости от лемм.

In [41]:
def lemmatisation(row):
    return m.lemmatize(row['purpose']) 
 
borrowers['lemmas'] = borrowers.apply(lemmatisation, axis = 1)

In [42]:
borrowers['lemmas']

0                             [покупка,  , жилье, \n]
1                   [приобретение,  , автомобиль, \n]
2                             [покупка,  , жилье, \n]
3                [дополнительный,  , образование, \n]
4                           [сыграть,  , свадьба, \n]
                             ...                     
21520                  [операция,  , с,  , жилье, \n]
21521               [сделка,  , с,  , автомобиль, \n]
21522                              [недвижимость, \n]
21523    [на,  , покупка,  , свой,  , автомобиль, \n]
21524             [на,  , покупка,  , автомобиль, \n]
Name: lemmas, Length: 21525, dtype: object

In [43]:
def purpose_briefly(row):
    purpose = row['lemmas']
    try:
        if 'жилье' in purpose or 'недвижимость' in purpose:
            return 'недвижимость'
        if 'свадьба' in purpose:
            return 'свадьба'
        if 'автомобиль' in purpose:
            return 'автомобиль'
        if 'образование' in purpose:
            return 'образование'
    except:
        print('Не присвоена цель заемщику. Проверьте корректность внесения данных')
        print(row)


In [44]:
borrowers['purpose_briefly'] = borrowers.apply(purpose_briefly, axis = 1)

In [45]:
borrowers['purpose_briefly']

0        недвижимость
1          автомобиль
2        недвижимость
3         образование
4             свадьба
             ...     
21520    недвижимость
21521      автомобиль
21522    недвижимость
21523      автомобиль
21524      автомобиль
Name: purpose_briefly, Length: 21525, dtype: object

Проверим, что для каждого заемщика была определена из четырех целей кредитования (суммарное количество строк по катергориям совпадает с общим числом строк таблицы, отсутствует сигнал "не присвоена цель" (указан в конструкции **try-except** функции **purpose_briefly**)):

In [46]:
borrowers['purpose_briefly'].value_counts()

недвижимость    10840
автомобиль       4315
образование      4022
свадьба          2348
Name: purpose_briefly, dtype: int64

### Вывод

Изучение столбца **purpose** показало, что среди значительного количества целей кредитования, записанных по-разному, можно выделить всего четыре, отличающихся друг от друга по смыслу: кредит на недвижимость, автомобиль, свадьбу или образование. С помощью лемматизации каждому из заемщиков присвоена одна из указанных целей.

### Категоризация данных <a name="categorization"></a>

В предыдущем пункте вместе с лемматизацией данных из столбца **purpose** была проведена и их категоризация. Для удобства интерпретации имеет смысл также категоризировать данные в столбце **children**, разбив их на три группы: нет детей, 1-2 ребенка, многодетный. Но прежде проверим корректность значений указанного столбца:

In [47]:
borrowers['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В данных определенно содержатся ошибки: число детей не может быть отрицательным. Также сомнителен показатель, равный 20. 

Заменим -1 на 1 (ошибка могла появиться, если данные набирали вручную и случайно поставили минус). Один ребенок в семье довольно типичен для выборки (более 20% заемщиков), поэтому увеличение этого показателя менее, чем на 1% сильно не исказит картину.

In [48]:
borrowers.loc[borrowers['children'] == -1, 'children'] = 1

Рассмотрим, что представляют из себя заемщики с 20 детьми:

In [49]:
borrowers.loc[borrowers['children'] == 20].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,purpose_briefly
606,20,880,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья,"[покупка, , жилье, \n]",недвижимость
720,20,855,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости,"[покупка, , недвижимость, \n]",недвижимость
1074,20,3310,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования,"[получение, , образование, \n]",образование
2510,20,2714,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью,"[операция, , с, , коммерческий, , недвижимо...",недвижимость
2941,20,2161,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля,"[на, , покупка, , автомобиль, \n]",автомобиль
3302,20,4641,35,среднее,1,Не женат / не замужем,4,F,госслужащий,0,167422.302208,профильное образование,"[профильный, , образование, \n]",образование
3396,20,4641,56,высшее,0,женат / замужем,0,F,компаньон,0,167422.302208,высшее образование,"[высокий, , образование, \n]",образование
3671,20,913,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,101255.492076,на покупку подержанного автомобиля,"[на, , покупка, , подержать, , автомобиль, \n]",автомобиль
3697,20,2907,40,среднее,1,гражданский брак,1,M,сотрудник,0,115380.694664,на покупку подержанного автомобиля,"[на, , покупка, , подержать, , автомобиль, \n]",автомобиль
3735,20,805,26,высшее,0,Не женат / не замужем,4,M,сотрудник,0,137200.646181,ремонт жилью,"[ремонт, , жилье, \n]",недвижимость


Закономерности обнаружить не удается. Более того, сложно предположить, откуда появилась такая ошибка, поэтому пренебрежем ею (всего 76 строк в массиве более чем из 21000 записей) и заменим 20 на самое часто встречаемое значение - 0.

In [50]:
borrowers.loc[borrowers['children'] == 20, 'children'] = 0

После преобразований некорректные значения ушли:

In [51]:
borrowers['children'].value_counts()

0    14225
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Теперь категоризируем заемщиков в зависимости от количества детей:

In [52]:
def num_children(row):
    num_child = row['children']
    try:
        if num_child == 0:
            return 'нет детей'
        if num_child == 1 or num_child == 2:
            return '1-2 ребенка'
        if num_child >= 0:
            return 'многодетный'
    except:
        print('Ошибка в исходных значениях')
        print(row)

In [53]:
borrowers['num_children'] = borrowers.apply(num_children, axis = 1)

In [54]:
borrowers['num_children'].value_counts()

нет детей      14225
1-2 ребенка     6920
многодетный      380
Name: num_children, dtype: int64

В задании также необходимо выявить зависимость между уровнем дохода и возвратом кредита в срок, поэтому желательно категоризировать заемщиков по уровню доходов. Ранее мы уже проверяли, в каком диапазоне лежит этот показатель:

In [55]:
borrowers['total_income'].min()

20667.26379327158

In [56]:
borrowers['total_income'].max()

2265604.028722744

Возьмем за основу следующую градацию:
- до 50 000 (вкл.) - очень низкий
- свыше 50 000 до 100 000 (вкл.) - низкий
- свыше 100 000 до 150 000 (вкл.) - средний
- свыше 150 000 до 200 000 (вкл.) - высокий
- свыше 200 000 - очень высокий

In [57]:
def income_grade(row):
    income_gr = row['total_income']
    try:
        if 0 <= income_gr <= 50000:
            return 'очень низкий доход (0 - 50 000)'
        if 50000 < income_gr <= 100000:
            return 'низкий доход (50 000 - 100 000)'
        if 100000 < income_gr <= 150000:
            return 'средний доход (100 000 - 150 000)'
        if 150000 < income_gr <= 200000:
            return 'высокий доход (150 000 - 200 000)'
        if 200000 < income_gr:
            return 'очень высокий доход (более 200 000)'
    except:
        print('Ошибка в исходных значениях дохода')
        print(row)

In [58]:
borrowers['income_grade'] = borrowers.apply(income_grade, axis = 1)

In [59]:
borrowers['income_grade'].value_counts()

высокий доход (150 000 - 200 000)      6292
средний доход (100 000 - 150 000)      5704
очень высокий доход (более 200 000)    5066
низкий доход (50 000 - 100 000)        4091
очень низкий доход (0 - 50 000)         372
Name: income_grade, dtype: int64

### Вывод

Данные категоризированы следующим образом: 
- **цель кредитования**:
    - недвижимость
    - образование
    - автомобиль
    - свадьба
- **количество детей**:
    - нет детей
    - 1-2 ребенка
    - многодетные
- **среднемесячный доход**:
    - очень низкий доход (до 50 000)
    - низкий доход (50 000 - 100 000)
    - средний доход (100 000 - 150 000)
    - высокий доход (150 000 - 200 000)
    - очень высокий доход (более 200 000)

### Ответы на вопросы<a name="answers"></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Поскольку последовательность действий, которую надо осуществить над данными для получения ответов на последние четыре вопроса, однотипна, составим функцию. 

На вход функция принимает датафрейм **data**, столбец с категориями **ind**, а также столбец **val**, по значениям которого строится сводная таблица (аналоги аргументов **index** и **values** для построения сводной таблицы).

Функция возвращает сводную таблицу, в строках которой находятся заемщики, разбитые по заданной категроии. Столбец **debt** будет показывать количество заемщиков с просрочкой по каждой категории, **total** - общее количество заемщиков указанной категории, **ratio** - долю заемщиков с просрочкой от общего их числа. Таблица отсортирована по столбцу **ratio** в порядке убывания.

In [60]:
def build_pivot_sum_debt(data, ind, val):
    try:
        pivot_t = data.pivot_table(index = ind, values = val, aggfunc = 'sum')
        pivot_t['total'] = data[ind].value_counts()
        pivot_t['ratio'] = pivot_t[val] / pivot_t['total']
        return pivot_t.sort_values('ratio', ascending = False)
    except:
        print('Проверьте аргументы функции')

Для ответа на первый вопрос составим сводную таблицу, которая будет отражать число заемщиков с просрочками по кредиту в разбивке по категориям показателя количества детей:

In [61]:
build_pivot_sum_debt(borrowers, 'num_children', 'debt')

,debt,total,ratio
num_children,,,
1-2 ребенка,639,6920,0.092341
многодетный,31,380,0.081579
нет детей,1071,14225,0.075290


### Вывод

Доля должников среди бездетных людей ниже, чем среди тех, у кого есть дети. Максимальный процент должников (9,2%) среди людей, у которых 1-2 ребенка. Среди многодетных - 8,1%. Доля должников среди бездетных самая низкая и составляет 7,5%.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [62]:
build_pivot_sum_debt(borrowers, 'family_status', 'debt')

,debt,total,ratio
family_status,,,
Не женат / не замужем,274,2813,0.097405
гражданский брак,388,4177,0.092890
женат / замужем,931,12380,0.075202
в разводе,85,1195,0.071130
вдовец / вдова,63,960,0.065625


### Вывод

Есть связь между наличием задолженности и семейным положением заемщика. Доля должников среди неженатых/незамужних самая высокая - 9,7%. Также она высока среди проживающих в гражданском браке - 9,3%. Среди женатых этот показатель составляет 7,5%, среди разведенных - 7,1%. Самая низкая - среди вдовцов (6,6%).

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [63]:
build_pivot_sum_debt(borrowers, 'income_grade', 'debt')

,debt,total,ratio
income_grade,,,
средний доход (100 000 - 150 000),491,5704,0.086080
высокий доход (150 000 - 200 000),538,6292,0.085505
низкий доход (50 000 - 100 000),331,4091,0.080909
очень высокий доход (более 200 000),358,5066,0.070667
очень низкий доход (0 - 50 000),23,372,0.061828


### Вывод

Самая высокая доля просрочки среди заемщиков с доходом от 100 000 до 150 000 - 8,6%, практически такое же значение среди заемщиков с доходами от 150 000 до 200 000 р. - 8,5%. Чуть лучше дело обстоит у людей с доходами от 50 000 до 100 000 (8%). Наилучшая платежная дисциплина у заемщиков с доходами свыше 200 000 (7%) и ниже 50 000 (6,1%).

- Как разные цели кредита влияют на его возврат в срок?

In [64]:
build_pivot_sum_debt(borrowers, 'purpose_briefly', 'debt')

,debt,total,ratio
purpose_briefly,,,
автомобиль,403,4315,0.093395
образование,370,4022,0.091994
свадьба,186,2348,0.079216
недвижимость,782,10840,0.072140


### Вывод

Максимальная доля неплательщиков по кредитам на приобретение автомобиля - 9,3%, также высоко значение по кредитам на образование - 9,2%. Заемные средства на свадьбу не возвращает 7,9% клиентов банка. Минимальный показатель по кредитам, связанным с недвижимостью, - 7,2%.

### Общий вывод <a name="conclusion"></a>

После проведенного исследования можно выделить проблемных заемщиков в каждой рассмотренной категории. 

Так, доля должников среди людей, имеющих детей, выше, чем среди бездетных. Максимальный процент должников (9,2%) составляют заемщики, у которых 1-2 ребенка. Среди многодетных таковых 8,1%. Доля должников среди бездетных самая низкая - 7,5%.

Имеет место связь между просрочками и семейным положением заемщика. Доля должников среди неженатых/незамужних самая высокая - 9,7%. Также она высока среди проживающих в гражданском браке - 9,3%. Среди женатых этот показатель составляет 7,5%, среди разведенных - 7,1%. Меньше всего должников среди вдовцов (6,6%).

У заемщиков с доходом от 100 000 до 150 000 самая значительная доля просрочки - 8,6%, практически такое же значение среди заемщиков с доходами от 150 000 до 200 000 р. - 8,5%. Чуть лучше дело обстоит у людей с доходами от 50 000 до 100 000 (8%). Наилучшая платежная дисциплина у заемщиков с доходами свыше 200 000 (7%) и ниже 50 000 (6,1%).

Больше всего неплательщиков среди взявших кредит на приобретение автомобиля - 9,3%, а также на образование - 9,2%. Заемные средства на свадьбу возвращает с просрочкой 7,9% клиентов банка. Наилучший показатель по кредитам, связанным с недвижимостью, - 7,2%.

Таким образом, можно сформировать портрет наиболее проблемного заемщика: это клиент, имеющий одного или двух детей, неженатый, со средним доходом 100 000 - 150 000, берущий кредит на приобретение автомобиля.

Банку-заказчику можно порекомендовать при скоринге клиентов делать выбор при прочих равных в пользу клиентов со следующими характеристиками:

- бездетные
- вдовцы или разведенные
- с доходами менее 50 000 либо более 200 000
- берущие кредит на операции с недвижимостью.